# 1. Code explaining lab_1.py 
## 1.1 faza testów 02.10.2024 22:99

Przed eksperymentem sprawdzałem model na danych losowych, liczyło się dla mnie jedynie czy  model działa (uruchamia się) i czy 'coś' z niego wychodzi. ![zdjęcie 1](image1.png) to był wykres dany po wylosowaniu 1szego zestawu danych.
```python
# Generowanie losowych danych
np.random.seed(42)  # Ustawienie ziarna losowości
num_samples = 1000 # Liczba próbek

# Generowanie losowych temperatur (270-400K) i zmian objętości (0.1-1.0)
temperatures = np.random.uniform(270, 400, num_samples)
delta_volumes = np.random.uniform(0.1, 1.0, num_samples)
```
wszystko bazowało na całkowicie losowych danych niezgodnych z realnymi pomiarami wiec nie ma co sie dziwić ze otrzymane informacje nie pozwalały na stwierdzenie niczego wiecej jak "no działa". 
![zdjęcie 2](image2.png)



# 2. Mój kod. 
Na tym etapie będe tłumaczył krok po kroku jak zbudowany jest mój kod i w jaki sposób będe go wykoprzystywał. Staram się by było jak naprościej ale jestem świadomy że jakaś minimalna wiedza z programowania w python jest potrzebna 
## 1.2 Import bibliotek

In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns

Tutaj jedynie przywołujemy najpotrzebniejsze biblioteki i zapisujemy je w po przywołaniu w kodzie nie musieć pisać całych nazw a jedynie skróty tj. `nn`, `pd` itd. 

## 1.2 Importowanie danych z pliku xlsx
Obok zapisu `.csv`, `.xlsx` jest podstawowym i standardowym zapisem plików excel. Ale w przeciwieństwie do `.csv` możemy importować wartości z kolumn (np.temperature) a nie musimy zapisywać danych w jednej kolumn i oddzielać jej przecinkami tak jak by muisiało to być w pliku `.csv`. 



In [7]:
# 0. Otwieranie danych
file_path = 'lab_1_dane.xlsx'
# Sprawdzenie, czy plik istnieje
if os.path.exists(file_path):
    # odczytanie danych z arkusza 
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    print(df.head())
else:
    print(f"Plik {file_path} nie istnieje. Upewnij się, że plik znajduje się w odpowiednim folderze.")


   temperature  delta_volume
0   318.690215      0.972626
1   393.592860      0.797620
2   365.159212      0.945549
3   347.825603      0.905345
4   290.282423      0.638110


Nie ma tu dużo do tłumacznia ale i tak postaram się zapisać najważniejsze informacje tak dla przypomnienia ;). 
- `file_path` - zapisujemy ścieżkę naszego pliku z którego bierzemy dane. Ważne że gdy plik znajduje się w tym samym folderze co kod, nie musimy niż więcej dopisywać po za nazwą. 
- tutaj wykonujemy polecenie by program sprawdził czy dany plik w danym miejscu istnieje
    ```python
    if os.path.exists(file_path):
    ```

    - Jeśli plik został znaleziony, sprawdzamy czy dane zostaly wczytane poprawnie. do tego używamy `.head()` ta funkcja wyciąga 5 pierwszym elementów z naszego pliku. 
        ```python
        df = pd.read_excel(file_path, sheet_name='Sheet1')
        # w zmiennej df zapisujemy nasze informacje z pliku `lab_1_dane.xlsx`
        print(df.head())
        ```
    - Jeśli plik nie został znaleziony wyskoczy nam informacje zwrotna:
        ```python
        else:
        print(f"Plik {file_path} nie istnieje. Upewnij się, że plik znajduje się w odpowiednim folderze.")
        ```

## 2.2 Zapisywanie danych jako zmienne i ich konwersja na tensory. 
Teraz jak już mamy otworzone dane w zmiennej możemy rozdzielić je na wynyki pomiaru tempreatury z kolumny `temperature` i zmiany objętości z kolumny `delta_volume`. 
### 2.2.1 Wyciąganie danych

In [8]:
# Przykładowe dane: temperatury (T) i odpowiadające im zmiany objętości (delta_V)
temperatures = df['temperature'].values  # Temperatura w Kelvinach
delta_volumes = df['delta_volume'].values  # Zmiana objętości



Przykładowo w zmiennej `temperatures` chcemny zapisać pomiary temperatury które aktualnie znajdują się w zmiennej `df` by to zrobić musimy wskazać jak się nazywa kolumna pod którą są szukane przez nas wartości. 
W tym przypadku jest to `temperatures` więc całość kodu wygląda następująco
```python
nasza_nowa_zmienna = zmienna_z_wszyskim['nazwa_szukanej_kolumny'].values 
#bo potrzebujemy wartości z tej kolumny. 
```

```python
temperatures = df['temperature'].values  # Temperatura w Kelvinach
```

I analogicznie lecimy dalej w celu "wyciągniecia" wszystkich potrzebnych przez nas informacji. 




### 2.2.2 Zapisywanie danych jako tensory. 
By użyć biblioteki `pytorch` do budowania modelu sieci neuronowej. Najpierw sobie trzeba wytłumaczyć "po co to komu ten tensor?"





#### 2.2.2.BONUS  po co to komu ten tensor?
Tensor to wielowymiarowa tablica danych, która jest uogólnieniem skalara, wektora i macierzy, taka macierz 3d (ale może mieć więcej niż 3 wymiary.)


Przykłady tensorów:
- Skalar: Tensor zerowego rzędu, pojedyncza liczba. $x \in \mathbb{R}$
- Wektor: Tensor pierwszego rzędu, jednowymiarowa tablica. $\mathbf{x} \in \mathbb{R}^n$ 
- Macierz: Tensor drugiego rzędu, dwuwymiarowa tablica. $\mathbf{X} \in \mathbb{R}^{m \times n}$ 
- Tensor wyższego rzędu: Wielowymiarowa tablica.  $\mathcal{X} \in \mathbb{R}^{d_1 \times d_2 \times \ldots \times d_k}$ 

```python
import torch
# Skalar
scalar = torch.tensor(3.14)
# Wektor
vector = torch.tensor([1.0, 2.0, 3.0])
# Macierz
matrix = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
# Tensor trzeciego rzędu
tensor = torch.tensor([[[1.0, 2.0], [3.0, 4.0]], [[5.0, 6.0], [7.0, 8.0]]])
print("Scalar:", scalar)
print("Vector:", vector)
print("Matrix:", matrix)
print("Tensor:", tensor)
```

**Dlaczego tensory są ważne?**
- Wielowymiarowość: Tensory mogą reprezentować dane w wielu wymiarach, co jest kluczowe w modelowaniu złożonych struktur danych.
- Operacje algebraiczne: Tensory umożliwiają wykonywanie operacji algebraicznych, takich jak iloczyn tensorowy, które są fundamentalne w algorytmach uczenia maszynowego.
- Efektywność obliczeniowa: Biblioteki takie jak TensorFlow i PyTorch są zoptymalizowane do pracy z tensorami, co pozwala na szybkie i efektywne przetwarzanie danych na GPU.
- Automatyczne różniczkowanie: Tensory są używane do obliczania gradientów w procesie optymalizacji i minimalizowania błędu predykcji modelu. 